# Data Combiner

Here we try to bring together all the data into one clean dataframe

In [1]:
# !pip install pandas_profiling
# !pip install ipywidgets

In [2]:
import pandas as pd
import os
import plotly.express as px
import numpy as np

In [3]:
from pandas_profiling import ProfileReport
from statistics import mean
from scipy.stats import f_oneway


In [4]:
result = pd.read_csv(r"..\Data\Resultate\combined_results.csv", index_col=0)
result_normal = result[result["Report Type"]=="Normal Report"]

In [5]:
result_normal[["Zero Shot Learning Sentence Raw", "Zero Shot Learning Sentence Mean", "Zero Shot Learning Sentence Mean Threshold"]].head()

,Zero Shot Learning Sentence Raw,Zero Shot Learning Sentence Mean,Zero Shot Learning Sentence Mean Threshold
0,"[0.0063936966471374035, 0.1543944627046585, 0....",0.086,0.135
1,"[0.22032485902309418, 0.21426941454410553, 0.1...",0.161,0.207
2,"[0.7101063132286072, 0.41432586312294006, 0.50...",0.597,0.597
3,"[0.005319107323884964, 0.049131620675325394, 0...",0.044,0.118
4,"[0.057856060564517975, 0.16787868738174438, 0....",0.112,0.164


In [6]:
result_normal[0:1]

,Company,Label,Zero Shot Fulltext,Sentence Cosine Similarity Wiki Summarized,Sentence Cosine Similarity Wiki Summarized Mean,Sentence Cosine Similarity Wiki Summarized Mean Threshold 0,Sentence by Sentence Cosine Similarity human rights,Sentence by Sentence Cosine Similarity human rights mean,Sentence by Sentence Cosine Similarity human rights Mean Threshold 0,Zero Shot Learning Sentence Raw,Zero Shot Learning Sentence Mean,Zero Shot Learning Sentence Mean Threshold,PDF Name,TF-IDF,Top N Words,Report Type
0,abb_sustainability_performance,social issues,0.249538,"[tensor([[0.1108]]), tensor([[0.0350]]), tenso...",0.071,0.104,"[tensor([[0.1108]]), tensor([[0.0350]]), tenso...",0.071,0.104,"[0.0063936966471374035, 0.1543944627046585, 0....",0.086,0.135,abb-group-sustainability-performance-report-2015,"[('performance', 0.25703375969085324), ('susta...","[('abb', 242), ('sustainability', 218), ('perf...",Normal Report


### 5.1.4	Sentence-BERT Similarität satzweise vs. Zero-Shot Learning Satzweise

In [21]:
result_normal_prep = result_normal[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Learning Sentence Mean Threshold"]]
result_prep_melt = result_normal_prep.melt(var_name="Data")
fig = px.box(result_prep_melt, y="value", points="all", color = "Data")
fig.show()



In [16]:
fig = px.scatter(result_normal, x="Zero Shot Learning Sentence Mean Threshold", y="Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", trendline="ols")
fig.show()
results = px.get_trendline_results(fig)
print(results)
results.px_fit_results.iloc[0].summary()


                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     738.6
Date:                Wed, 14 Dec 2022   Prob (F-statistic):          2.48e-103
Time:                        15:57:20   Log-Likelihood:                 1153.4
No. Observations:                 545   AIC:                            -2303.
Df Residuals:                     543   BIC:                            -2294.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0621      0.002     29.310      0.000       0.058       0.066
x1             0.1932      0.007     27.178      0.000       0.179       0.207
==============================================================================
Omnibus:                      158.419   Durbin-Watson:                   1.331
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1054.494
Skew:                           1.099   Prob(JB):                    1.05e-229
Kurtosis:                       9.450   Cond. No.                         6.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
result_prep = result[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Fulltext", "Label"]]

croped_results = result_prep[result_prep["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] <= 0.179]
croped_results = croped_results[croped_results["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] >= 0.032]
croped_results = croped_results[croped_results["Zero Shot Fulltext"] <= 0.373]


In [12]:
result_prep = croped_results[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Fulltext"]]
result_prep_melt = result_prep.melt(var_name="Data")
fig = px.box(result_prep_melt, y="value", points="all", color = "Data")
fig.show()



In [13]:
fig = px.scatter(croped_results, x="Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", y="Zero Shot Fulltext", trendline="ols", color = "Label")
fig.show()
results = px.get_trendline_results(fig)
print(results)

            Label                                     px_fit_results
0   social issues  <statsmodels.regression.linear_model.Regressio...
1    human rights  <statsmodels.regression.linear_model.Regressio...
2      labour law  <statsmodels.regression.linear_model.Regressio...
3           fraud  <statsmodels.regression.linear_model.Regressio...
4  sustainability  <statsmodels.regression.linear_model.Regressio...


In [14]:
results.px_fit_results.iloc[0].summary()
result_prep_melt_threshold = result[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] ]
result_prep_melt_no_threshold = result[["Sentence Cosine Similarity Wiki Summarized Mean"] ]
result_prep_melt_no_threshold[0:1]
f_oneway(result_prep_melt_threshold, result_prep_melt_no_threshold)


F_onewayResult(statistic=array([173.5738793]), pvalue=array([4.46097502e-37]))